In [0]:
# LOADING THE BRONZE ACCOUNTS TABLE
bronze_accounts_df = spark.table("finance_fraudworkspace.bronze_managed.accounts_bronze")

In [0]:
# CLEAN ACCOUNTS DATA
from pyspark.sql.functions import col, to_timestamp, to_date

clean_accounts_df = (
    bronze_accounts_df
    .withColumn("Updated_ts", to_timestamp(col("Updated_ts"), "dd-MM-yyyy HH:mm"))
)

In [0]:
from pyspark.sql.functions import col, when, upper, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# REMOVE NULL ACCOUNTS IDs
accounts_silver = clean_accounts_df.filter(col("Account_id").isNotNull())

# STANDARDIZE ACCOUNT STATUS
accounts_silver = accounts_silver.withColumn(
    "Account_status",
    upper(col("Account_status"))
)

# FIX NULL CREDIT LIMIT
accounts_silver = accounts_silver.withColumn(
    "Credit_limit",
    when(col("Credit_limit").isNull(), 1000)
    .otherwise(col("Credit_limit").cast(IntegerType()))
)

# REMOVE DUPLICATES
window_spec = Window.partitionBy("Account_id").orderBy(col("Updated_ts").desc())

accounts_silver = (
    accounts_silver
        .withColumn("row_num", row_number().over(window_spec))
        .filter(col("row_num") == 1)
        .drop("row_num")
)

# VALIDATE CUSTOMERS 
from pyspark.sql.functions import trim, upper, col

# Clean accounts side
accounts_silver = accounts_silver.withColumn(
    "Customer_id",
    upper(trim(col("Customer_id")))
)

# Clean customers dimension side
customers_dim = spark.table("finance_fraudworkspace.silver.dim_customers") \
    .withColumn("Customer_id", upper(trim(col("Customer_id"))))

accounts_silver = accounts_silver.join(
    customers_dim.select("Customer_id").distinct(),
    on="Customer_id",
    how="inner"
)

display(accounts_silver)

Customer_id,Account_id,Account_status,Credit_limit,Updated_ts
C001,A001,ACTIVE,10000,2024-01-01T00:00:00Z
C002,A002,BLOCKED,5000,2024-02-15T00:00:00Z
C003,A003,BLOCKED,3000,2024-01-01T00:00:00Z
C004,A004,ACTIVE,1000,2024-01-01T00:00:00Z
C005,A005,ACTIVE,7000,2024-01-01T00:00:00Z


In [0]:
silver_accounts_clean = "finance_fraudworkspace.silver_managed.accounts_silver"

accounts_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(silver_accounts_clean)

display(silver_accounts_clean)

'finance_fraudworkspace.silver_managed.accounts_silver'